In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import pandas as pd
df=pd.read_csv("../input/online-payments-fraud-detection-dataset/PS_20174392719_1491204439457_log.csv")

In [16]:
df.head()

In [17]:
import numpy as np
train, validation, test = np.split(df.sample(frac=1), [int(0.6*len(df)),int(0.8*len(df))])

In [18]:
len(train)

In [19]:
type_new = pd.get_dummies(df['type'],drop_first=True)
df = pd.concat([df,type_new],axis=1)
df.head()
X = df[['step','CASH_OUT','DEBIT','PAYMENT','TRANSFER','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud']]
y = df['isFraud']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
LogisticRegression()
pred_lr = lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_lr))

In [22]:
len(X_test)

In [23]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=7,shuffle=True)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

In [24]:
df['Gender'] = np.random.choice(['Male', 'Female'], size=len(df), p=[0.6, 0.4])

ownership_filter = df['type'].isin(['CASH_IN', 'PAYMENT','CASH_OUT','TRANSFER','DEBIT'])
df = df.drop(df[~ownership_filter].index)

(df['type'].value_counts() / len(df)).sort_values(ascending=False), (df['Gender'].value_counts() / len(df)).sort_values(ascending=False)

In [25]:
df['Stratify'] = df['type'] + ", " + df['Gender']
(df['Stratify'].value_counts() / len(df)).sort_values(ascending=False)

In [26]:
def stratify_data(df_data, stratify_column_name, stratify_values, stratify_proportions, random_state=None):
    """Stratifies data according to the values and proportions passed in

    Args:
        df_data (DataFrame): source data
        stratify_column_name (str): The name of the single column in the dataframe that holds the data values that will be used to stratify the data
        stratify_values (list of str): A list of all of the potential values for stratifying e.g. "Male, Graduate", "Male, Undergraduate", "Female, Graduate", "Female, Undergraduate"
        stratify_proportions (list of float): A list of numbers representing the desired propotions for stratifying e.g. 0.4, 0.4, 0.2, 0.2, The list values must add up to 1 and must match the number of values in stratify_values
        random_state (int, optional): sets the random_state. Defaults to None.

    Returns:
        DataFrame: a new dataframe based on df_data that has the new proportions represnting the desired strategy for stratifying
    """
    df_stratified = pd.DataFrame(columns = df_data.columns) # Create an empty DataFrame with column names matching df_data

    pos = -1
    for i in range(len(stratify_values)): # iterate over the stratify values (e.g. "Male, Undergraduate" etc.)
        pos += 1
        if pos == len(stratify_values) - 1: 
            ratio_len = len(df_data) - len(df_stratified) # if this is the final iteration make sure we calculate the number of values for the last set such that the return data has the same number of rows as the source data
        else:
            ratio_len = int(len(df_data) * stratify_proportions[i]) # Calculate the number of rows to match the desired proportion

        df_filtered = df_data[df_data[stratify_column_name] ==stratify_values[i]] # Filter the source data based on the currently selected stratify value
        df_temp = df_filtered.sample(replace=True, n=ratio_len, random_state=random_state) # Sample the filtered data using the calculated ratio
        
        df_stratified = pd.concat([df_stratified, df_temp]) # Add the sampled / stratified datasets together to produce the final result
        
    return df_stratified # Return the stratified, re-sampled data 

In [27]:
stratify_values = ['CASH_OUT, Male', 'PAYMENT, Male', 'CASH_OUT, Female', 'PAYMENT, Female','CASH_IN, Male','CASH_IN, Female','TRANSFER, Male','TRANSFER, Female','DEBIT, Male','DEBIT, Female']
stratify_proportions = [0.2,0.2,0.05,0.05,0.05,0.05,0.1,0.1,0.1,0.1]
df_stratified = stratify_data(df, 'Stratify', stratify_values, stratify_proportions, random_state=42)
df_stratified

In [ ]:
C = [10, 1, .1, .001]

for c in C:
    clf = LogisticRegression(penalty='l1', C=c, solver='liblinear')
    clf.fit(X_train, y_train)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_std, y_train))
    print('Test accuracy:', clf.score(X_test_std, y_test))
    print('')